In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns

from pandas.api.types import CategoricalDtype

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:24: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


In [7]:
# Load data
sociodemografico_cuenca = pd.read_csv(f'../../data/external/temporal_fromjupyter/arroyo_vivo/censoanalysis_for_arroyovivo_cuenca.csv')
sociodemografico_cuenca.drop(columns='Unnamed: 0',inplace=True)
sociodemografico_cuenca.rename(columns={'data':'data_cuenca',
                                        'wgt_%':'wgt_%_cuenca'},inplace=True)

# Show
print(sociodemografico_cuenca.shape)
sociodemografico_cuenca.head(2)

(74, 4)


,indicador,data_cuenca,wgt_%_cuenca,weight
0,POBTOT,104048.0,NaN,NaN
1,POBFEM,52327.0,50.291212,POBTOT


In [8]:
sociodemografico_poligono = pd.read_csv(f'../../data/external/temporal_fromjupyter/arroyo_vivo/censoanalysis_for_poligono_arroyovivo.csv')
sociodemografico_poligono.drop(columns='Unnamed: 0',inplace=True)
sociodemografico_poligono.rename(columns={'data':'data_poli',
                                        'wgt_%':'wgt_%_poli'},inplace=True)

# Show
print(sociodemografico_poligono.shape)
sociodemografico_poligono.head(2)

(74, 4)


,indicador,data_poli,wgt_%_poli,weight
0,POBTOT,97554.0,NaN,NaN
1,POBFEM,49160.0,50.392603,POBTOT


In [9]:
sociodemografico_buffer = pd.read_csv(f'../../data/external/temporal_fromjupyter/arroyo_vivo/censoanalysis_for_buffer_200mts.csv')
sociodemografico_buffer.drop(columns='Unnamed: 0',inplace=True)
sociodemografico_buffer.rename(columns={'data':'data_buffer',
                                        'wgt_%':'wgt_%_buffer'},inplace=True)

# Show
print(sociodemografico_buffer.shape)
sociodemografico_buffer.head(2)

(74, 4)


,indicador,data_buffer,wgt_%_buffer,weight
0,POBTOT,40524.0,NaN,NaN
1,POBFEM,20489.0,50.560162,POBTOT


In [23]:
# Merge cuenca with poligono
merged1 = sociodemografico_cuenca.merge(sociodemografico_poligono[['indicador','data_poli','wgt_%_poli']], on='indicador')

# Merge cuenca and poligono with buffer
merged2 = merged1.merge(sociodemografico_buffer[['indicador','data_buffer','wgt_%_buffer']],on='indicador')

# Reorder columns
merged2 = merged2[['indicador','weight','data_cuenca','wgt_%_cuenca','data_poli','wgt_%_poli','data_buffer','wgt_%_buffer']]

# Find zone of highest value for each row
comparing_values = ['wgt_%_cuenca','wgt_%_poli','wgt_%_buffer']

merged2['highest'] = merged2[comparing_values].idxmax(axis=1)

# Show
merged2.head(35)

,indicador,weight,data_cuenca,wgt_%_cuenca,data_poli,wgt_%_poli,data_buffer,wgt_%_buffer,highest
0,POBTOT,NaN,104048.0,NaN,97554.0,NaN,40524.0,NaN,NaN
1,POBFEM,POBTOT,52327.0,50.291212,49160.0,50.392603,20489.0,50.560162,wgt_%_buffer
2,POBMAS,POBTOT,51150.0,49.160003,47826.0,49.025155,19955.0,49.242424,wgt_%_buffer
3,P_0A2,POBTOT,2781.0,2.672805,2466.0,2.527831,1200.0,2.961208,wgt_%_buffer
4,P_0A2_F,POBFEM,1023.0,1.955014,911.0,1.853133,489.0,2.386646,wgt_%_buffer
5,P_0A2_M,POBMAS,1089.0,2.129032,931.0,1.946640,487.0,2.440491,wgt_%_buffer
6,P_3A5,POBTOT,3301.0,3.172574,3043.0,3.119298,1391.0,3.432534,wgt_%_buffer
7,P_3A5_F,POBFEM,1325.0,2.532154,1217.0,2.475590,565.0,2.757577,wgt_%_buffer
8,P_3A5_M,POBMAS,1336.0,2.611926,1232.0,2.576005,585.0,2.931596,wgt_%_buffer
9,P_6A11,POBTOT,6381.0,6.132746,5673.0,5.815241,2548.0,6.287632,wgt_%_buffer


In [27]:
# Show
merged2.tail(38)

,indicador,weight,data_cuenca,wgt_%_cuenca,data_poli,wgt_%_poli,data_buffer,wgt_%_buffer,highest
36,P3YM_HLI,P_3YMAS,1339.0,1.386502,1092.0,1.175103,620.0,1.589132,wgt_%_buffer
37,P3YM_HLI_F,P_3YMAS_F,662.0,1.353479,533.0,1.130675,323.0,1.632467,wgt_%_buffer
38,P3YM_HLI_M,P_3YMAS_M,427.0,0.896155,340.0,0.742764,201.0,1.045514,wgt_%_buffer
39,PCON_DISC,POBTOT,3252.0,3.125481,3066.0,3.142875,1249.0,3.082124,wgt_%_poli
40,PCON_LIMI,POBTOT,9218.0,8.859373,8953.0,9.177481,3717.0,9.172342,wgt_%_poli
41,P3A5_NOA,P_3A5,607.0,18.388367,600.0,19.717384,297.0,21.351546,wgt_%_buffer
42,P3A5_NOA_F,P_3A5_F,184.0,13.886792,196.0,16.105177,104.0,18.407080,wgt_%_buffer
43,P3A5_NOA_M,P_3A5_M,167.0,12.500000,166.0,13.474026,78.0,13.333333,wgt_%_poli
44,P6A11_NOA,P_6A11,136.0,2.131327,121.0,2.132910,68.0,2.668760,wgt_%_buffer
45,P6A11_NOAF,P_6A11_F,27.0,0.993377,21.0,0.887199,13.0,1.234568,wgt_%_buffer
